<a href="https://colab.research.google.com/github/Artyom995/WebSocet/blob/main/WebSocet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import websockets
import json
import numpy as np
from datetime import datetime, timedelta
from collections import deque
from sklearn.preprocessing import MinMaxScaler
from IPython.display import clear_output
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM
import threading

# Константы
SHORT_TERM_UPDATE_INTERVAL = 1.0  # Интервал обновления краткосрочного прогноза (секунды)
MID_TERM_UPDATE_INTERVAL = 60.0   # Интервал обновления среднесрочных прогнозов (секунды)
PING_INTERVAL = 20                # Интервал отправки пинга (в секундах)
RECONNECT_DELAY = 5               # Задержка перед переподключением (в секундах)
BUFFER_MAXLEN = 7000              # Максимальный размер буфера
TRAIN_INTERVAL = 180              # Интервал обучения моделей (в секундах)
HISTORY_LENGTH = 100              # Длина истории для моделей
PRICE_STEP = 0.1                  # Шаг округления цены для кластеризации

class MarketAnalyzer:
    def __init__(self, symbol):
        """Инициализация анализатора рынка."""
        self.symbol = symbol
        self.orderbook = {'asks': [], 'bids': []}
        self.market_data = deque(maxlen=BUFFER_MAXLEN)  # Хранит {'timestamp': datetime, 'price': float, 'volume': float}
        self.forecast_history = deque(maxlen=1000)
        self.ws = None
        self.is_connected = False
        self.last_ping = datetime.now()
        self.last_short_term_update = datetime.now()
        self.last_mid_term_1min_update = datetime.now()
        self.last_mid_term_2min_update = datetime.now()
        self.last_train_time = datetime.now()
        self.current_forecast = {
            'short_term': {},
            'mid_term_1min': {},
            'mid_term_2min': {}
        }
        self.scaler = MinMaxScaler()
        self.cnn_model = self.build_cnn_model()
        self.lstm_model = self.build_lstm_model()
        self.is_trained = False
        # Структуры для отслеживания точности
        self.short_term_pending_forecasts = []  # Ожидающие краткосрочные прогнозы
        self.mid_term_1min_pending_forecasts = []  # Ожидающие прогнозы на 1 минуту
        self.mid_term_2min_pending_forecasts = []  # Ожидающие прогнозы на 2 минуты
        self.short_term_accuracy_history = deque(maxlen=1000)  # История точности краткосрочных прогнозов
        self.mid_term_1min_accuracy_history = deque(maxlen=1000)  # История точности прогнозов на 1 минуту
        self.mid_term_2min_accuracy_history = deque(maxlen=1000)  # История точности прогнозов на 2 минуты
        self.delta = 0.4

    def build_cnn_model(self):
        """Создание CNN модели для краткосрочных прогнозов с 4 признаками."""
        model = Sequential()
        model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(20, 4)))  # 4 features: price, volume, orders, cumulative_volume
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dense(2, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def build_lstm_model(self):
        """Создание LSTM модели для среднесрочных прогнозов."""
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(HISTORY_LENGTH, 3)))  # 3 features: price, volume, orders
        model.add(LSTM(50))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(2, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    async def maintain_connection(self):
        while True:
            try:
                is_closed = True
                if self.ws is not None:
                    try:
                        is_closed = self.ws.closed
                    except AttributeError:
                        is_closed = True
                if not self.is_connected or is_closed:
                    await self.connect()
                await asyncio.sleep(1)
            except Exception as e:
                print(f"Ошибка поддержания соединения: {str(e)}")
                await asyncio.sleep(5)

    async def connect(self):
        """Установка соединения с WebSocket OKX."""
        try:
            self.ws = await websockets.connect(
                "wss://ws.okx.com:8443/ws/v5/public",
                ping_interval=20,
                ping_timeout=30,
                close_timeout=10
            )
            await self.ws.send(json.dumps({
                "op": "subscribe",
                "args": [{"channel": "books", "instId": self.symbol}]
            }))
            self.is_connected = True
            print("✅ Соединение установлено")
            await self.listen()
        except Exception as e:
            print(f"❗ Ошибка подключения: {str(e)}")
            self.is_connected = False
            self.ws = None
            await asyncio.sleep(RECONNECT_DELAY)

    async def listen(self):
        """Прослушивание сообщений от WebSocket."""
        try:
            async for msg in self.ws:
                await self.handle_message(msg)
        except websockets.ConnectionClosed as e:
            print(f"🔌 Соединение закрыто: {e.code} - {e.reason}")
            self.is_connected = False
        except Exception as e:
            print(f"⚠️ Ошибка чтения сообщений: {str(e)}")
            self.is_connected = False

    def update_orderbook(self, data):
        """Обновление книги ордеров."""
        def update_levels(side, new_levels):
            current = {float(l[0]): l for l in self.orderbook[side] if isinstance(l[0], (int, float))}
            for level in new_levels:
                try:
                    price = float(level[0])  # Convert price to float
                    volume = float(level[1])  # Convert volume to float
                except (ValueError, IndexError) as e:
                    print(f"Пропуск некорректного уровня: {level} - Ошибка: {str(e)}")
                    continue
                if volume > 0:
                    current[price] = [price, volume]
                else:
                    current.pop(price, None)
            self.orderbook[side] = sorted(
                current.values(),
                key=lambda x: x[0],
                reverse=(side == 'bids')
            )
        update_levels('asks', data.get('asks', []))
        update_levels('bids', data.get('bids', []))
        current_price = self.get_current_price()
        if current_price > 0:
            volume = (sum(level[1] for level in self.orderbook['bids'][:5]) +
                      sum(level[1] for level in self.orderbook['asks'][:5]))
            self.market_data.append({
                'timestamp': datetime.now(),
                'price': current_price,
                'volume': volume
            })

    def get_current_price(self):
        """Получение текущей средней цены."""
        if not self.orderbook['bids'] or not self.orderbook['asks']:
            return 0
        best_bid = float(self.orderbook['bids'][0][0])
        best_ask = float(self.orderbook['asks'][0][0])
        return (best_bid + best_ask) / 2

    def cluster_orderbook(self, price_step=PRICE_STEP):
        """Формирование кластеров по ценовым уровням с кумулятивным объемом."""
        clusters = {'bids': {}, 'asks': {}}
        for side in ['bids', 'asks']:
            for price, volume in self.orderbook[side]:
                if not isinstance(price, (int, float)):
                    print(f"Пропуск некорректной цены: {price} (тип: {type(price)})")
                    continue
                cluster_price = round(price / price_step) * price_step
                if cluster_price not in clusters[side]:
                    clusters[side][cluster_price] = {'volume': 0, 'orders': 0}
                clusters[side][cluster_price]['volume'] += float(volume)
                clusters[side][cluster_price]['orders'] += 1

        # Сортировка кластеров
        bid_clusters = sorted(clusters['bids'].items(), key=lambda x: x[0], reverse=True)  # От наибольшей цены
        ask_clusters = sorted(clusters['asks'].items(), key=lambda x: x[0])  # От наименьшей цены

        # Кумулятивный объем для bid (от наибольшей цены вниз)
        cumulative_bid_volume = 0
        for price, data in bid_clusters:
            cumulative_bid_volume += data['volume']
            data['cumulative_volume'] = cumulative_bid_volume

        # Кумулятивный объем для ask (от наименьшей цены вверх)
        cumulative_ask_volume = 0
        for price, data in ask_clusters:
            cumulative_ask_volume += data['volume']
            data['cumulative_volume'] = cumulative_ask_volume

        # Преобразование обратно в словари
        clusters['bids'] = {price: data for price, data in bid_clusters}
        clusters['asks'] = {price: data for price, data in ask_clusters}

        return clusters

    def prepare_cluster_data(self, current_price, num_levels=10):
        """Подготовка данных кластеров для модели, включая кумулятивный объем."""
        clusters = self.cluster_orderbook()
        bid_levels = sorted(clusters['bids'].keys(), reverse=True)[:num_levels]
        ask_levels = sorted(clusters['asks'].keys())[:num_levels]
        data = []
        for level in bid_levels:
            data.append([
                level,
                clusters['bids'][level]['volume'],
                clusters['bids'][level]['orders'],
                clusters['bids'][level]['cumulative_volume']
            ])
        for level in ask_levels:
            data.append([
                level,
                clusters['asks'][level]['volume'],
                clusters['asks'][level]['orders'],
                clusters['asks'][level]['cumulative_volume']
            ])
        return np.array(data)

    def train_models(self):
        """Обучение моделей CNN и LSTM на данных."""
        if len(self.market_data) < HISTORY_LENGTH:
            return

        # Обучение CNN
        X_cnn = []
        y_cnn = []
        for i in range(HISTORY_LENGTH, len(self.market_data) - 1):
            data = self.prepare_cluster_data(self.market_data[i]['price'])
            if data is None:
                continue
            X_cnn.append(data)
            price_i = self.market_data[i]['price']
            price_next = self.market_data[i + 1]['price']
            y_cnn.append([1, 0] if price_next > price_i else [0, 1])
        if X_cnn and y_cnn:
            X_cnn = np.array(X_cnn)
            y_cnn = np.array(y_cnn)
            self.cnn_model.fit(X_cnn, y_cnn, epochs=5, batch_size=32, verbose=0)

        # Обучение LSTM для среднесрочных прогнозов
        X_lstm = []
        y_lstm = []
        horizon = 60  # 60 секунд для горизонта прогноза (можно настроить)
        for i in range(HISTORY_LENGTH, len(self.market_data) - horizon):
            sequence = []
            for j in range(i - HISTORY_LENGTH, i):
                data = self.market_data[j]
                sequence.append([data['price'], data['volume'], 0])
            X_lstm.append(sequence)
            price_i = self.market_data[i]['price']
            price_future = self.market_data[i + horizon]['price']
            y_lstm.append([1, 0] if price_future > price_i else [0, 1])
        if X_lstm and y_lstm:
            X_lstm = np.array(X_lstm)
            y_lstm = np.array(y_lstm)
            self.lstm_model.fit(X_lstm, y_lstm, epochs=5, batch_size=32, verbose=0)
            self.is_trained = True

    def train_models_async(self):
        threading.Thread(target=self.train_models).start()

    def make_short_term_forecast(self, current_price):
        """Краткосрочный прогноз на следующую секунду на основе кластеров."""
        if not self.is_trained or len(self.market_data) < HISTORY_LENGTH:
            return {'prob_up': 0.5, 'prob_down': 0.5}
        data = self.prepare_cluster_data(current_price)
        if data is None:
            return {'prob_up': 0.5, 'prob_down': 0.5}
        data = np.expand_dims(data, axis=0)
        prediction = self.cnn_model.predict(data, verbose=0)[0]
        return {'prob_up': float(prediction[0]), 'prob_down': float(prediction[1])}

    def make_mid_term_forecast(self, current_price, minutes):
        """Среднесрочный прогноз на указанное количество минут (1 или 2).

        Аргументы:
            current_price (float): Текущая цена.
            minutes (int): Горизонт прогноза в минутах (1 или 2).

        Возвращает:
            dict: Словарь с ключами 'prob_upper', 'prob_lower', 'upper_target', 'lower_target'.
        """
        # Проверка на наличие обученной модели и достаточного количества данных
        if not self.is_trained or len(self.market_data) < HISTORY_LENGTH:
            return {
                'prob_upper': 0.5,
                'prob_lower': 0.5,
                'upper_target': current_price + (0.5 if minutes == 1 else 1),
                'lower_target': current_price - (0.5 if minutes == 1 else 1)
            }

        # Подготовка данных для LSTM: последние HISTORY_LENGTH записей
        X_lstm = []
        for i in range(-HISTORY_LENGTH, 0):
            data = self.market_data[i]
            X_lstm.append([data['price'], data['volume'], 0])  # Используем цену, объем и заглушку (0) для третьего признака
        X_lstm = np.array(X_lstm)
        X_lstm = np.expand_dims(X_lstm, axis=0)  # Преобразуем в форму (1, HISTORY_LENGTH, 3) для LSTM

        # Прогноз с помощью модели LSTM
        prediction = self.lstm_model.predict(X_lstm, verbose=0)[0]
        prob_upper = float(prediction[0])  # Вероятность роста
        prob_lower = float(prediction[1])  # Вероятность падения

        # Определение целевых цен в зависимости от minutes
        if minutes == 1:
            upper_target = current_price + 0.5
            lower_target = current_price - 0.5
        else:  # minutes == 2
            upper_target = current_price + 1
            lower_target = current_price - 1

        # Возвращаем результат в виде словаря
        return {
            'prob_upper': prob_upper,
            'prob_lower': prob_lower,
            'upper_target': upper_target,
            'lower_target': lower_target
        }

    def get_price_at(self, target_time):
        """Получение цены на заданное время."""
        if not self.market_data:
            return None
        closest_entry = min(self.market_data, key=lambda x: abs((x['timestamp'] - target_time).total_seconds()))
        return closest_entry['price']

    def check_accuracy(self):
        """Проверка точности прогнозов после истечения временного горизонта."""
        now = datetime.now()

        # Краткосрочный прогноз (1 секунда) — оставим как есть или добавим малый допуск
        matured = [f for f in self.short_term_pending_forecasts if (now - f['timestamp']).total_seconds() >= 1]
        for f in matured:
            price_at_t = self.get_price_at(f['timestamp'])
            target_time = f['timestamp'] + timedelta(seconds=1)
            price_after_1s = self.get_price_at(target_time)
            if price_at_t is not None and price_after_1s is not None:
                # Добавим небольшой допуск для краткосрочных прогнозов, например 0.01
                if f['predicted_direction'] == 'up':
                    is_correct = price_after_1s > price_at_t or abs(price_after_1s - price_at_t) < 0.01
                else:
                    is_correct = price_after_1s < price_at_t or abs(price_after_1s - price_at_t) < 0.01
                self.short_term_accuracy_history.append(is_correct)
        self.short_term_pending_forecasts = [f for f in self.short_term_pending_forecasts if f not in matured]

        # Среднесрочный прогноз на 1 минуту
        matured_1min = [f for f in self.mid_term_1min_pending_forecasts if (now - f['timestamp']).total_seconds() >= 60]

        for f in matured_1min:
            target_time = f['timestamp'] + timedelta(minutes=1)
            price_after_1min = self.get_price_at(target_time)
            if price_after_1min is not None:

                predicted_event = f['predicted_event']
                upper_target = f['upper_target']
                lower_target = f['lower_target']
                if predicted_event == 'upper':
                    # Рост: верен, если цена >= upper_target ИЛИ в диапазоне [upper_target - delta, upper_target]
                    is_correct = price_after_1min >= upper_target or price_after_1min >= upper_target - self.delta
                else:  # 'lower'
                    # Падение: верен, если цена <= lower_target ИЛИ в диапазоне [lower_target, lower_target + delta]
                    is_correct = price_after_1min <= lower_target or price_after_1min <= lower_target + self.delta
                self.mid_term_1min_accuracy_history.append(is_correct)
            else:
                print(f"Не удалось найти цену для времени {target_time}")
        self.mid_term_1min_pending_forecasts = [f for f in self.mid_term_1min_pending_forecasts if f not in matured_1min]

        # Среднесрочный прогноз на 2 минуты
        matured_2min = [f for f in self.mid_term_2min_pending_forecasts if (now - f['timestamp']).total_seconds() >= 120]
        for f in matured_2min:
            target_time = f['timestamp'] + timedelta(minutes=2)
            price_after_2min = self.get_price_at(target_time)
            if price_after_2min is not None:
                predicted_event = f['predicted_event']
                upper_target = f['upper_target']
                lower_target = f['lower_target']
                if predicted_event == 'upper':
                    # Рост: верен, если цена >= upper_target ИЛИ в диапазоне [upper_target - delta, upper_target]
                    is_correct = price_after_2min >= upper_target or price_after_2min >= upper_target - self.delta
                else:  # 'lower'
                    # Падение: верен, если цена <= lower_target ИЛИ в диапазоне [lower_target, lower_target + delta]
                    is_correct = price_after_2min <= lower_target or price_after_2min <= lower_target + self.delta
                self.mid_term_2min_accuracy_history.append(is_correct)
            else:
                print(f"Не удалось найти цену для времени {target_time}")
        self.mid_term_2min_pending_forecasts = [f for f in self.mid_term_2min_pending_forecasts if f not in matured_2min]

    def get_short_term_accuracy(self):
        """Расчет точности краткосрочных прогнозов в процентах."""
        if not self.short_term_accuracy_history:
            return 0.0
        return sum(self.short_term_accuracy_history) / len(self.short_term_accuracy_history) * 100

    def get_mid_term_1min_accuracy(self):
        """Расчет точности прогнозов на 1 минуту в процентах."""
        if not self.mid_term_1min_accuracy_history:
            return 0.0
        return sum(self.mid_term_1min_accuracy_history) / len(self.mid_term_1min_accuracy_history) * 100

    def get_mid_term_2min_accuracy(self):
        """Расчет точности прогнозов на 2 минуты в процентах."""
        if not self.mid_term_2min_accuracy_history:
            return 0.0
        return sum(self.mid_term_2min_accuracy_history) / len(self.mid_term_2min_accuracy_history) * 100

    async def handle_message(self, msg):
        """Обработка входящих сообщений от WebSocket."""
        try:
            now = datetime.now()
            if (now - self.last_ping).total_seconds() > PING_INTERVAL:
                await self.ws.send(json.dumps({"op": "ping"}))
                self.last_ping = now
            data = json.loads(msg)
            if 'event' in data and data['event'] == 'pong':
                return
            if 'data' in data:
                self.update_orderbook(data['data'][0])
                current_price = self.get_current_price()
                if current_price <= 0:
                    return
                if (now - self.last_train_time).total_seconds() > TRAIN_INTERVAL:
                    self.train_models_async()
                    self.last_train_time = now

                # Обновление краткосрочного прогноза (каждую секунду)
                if (now - self.last_short_term_update).total_seconds() >= SHORT_TERM_UPDATE_INTERVAL:
                    short_term_forecast = self.make_short_term_forecast(current_price)
                    self.current_forecast['short_term'] = short_term_forecast
                    self.last_short_term_update = now
                    predicted_direction = 'up' if short_term_forecast['prob_up'] > short_term_forecast['prob_down'] else 'down'
                    self.short_term_pending_forecasts.append({
                        'timestamp': now,
                        'predicted_direction': predicted_direction
                    })
                    self.print_all_analyses(current_price)

                # Обновление среднесрочного прогноза на 1 минуту (каждую минуту)
                if (now - self.last_mid_term_1min_update).total_seconds() >= MID_TERM_UPDATE_INTERVAL:
                    mid_term_1min_forecast = self.make_mid_term_forecast(current_price, 1)
                    self.current_forecast['mid_term_1min'] = mid_term_1min_forecast
                    self.last_mid_term_1min_update = now
                    predicted_event = 'upper' if mid_term_1min_forecast['prob_upper'] > mid_term_1min_forecast['prob_lower'] else 'lower'
                    self.mid_term_1min_pending_forecasts.append({
                        'timestamp': now,
                        'predicted_event': predicted_event,
                        'upper_target': mid_term_1min_forecast['upper_target'],
                        'lower_target': mid_term_1min_forecast['lower_target']
                    })

                # Обновление среднесрочного прогноза на 2 минуты (каждую минуту)
                if (now - self.last_mid_term_2min_update).total_seconds() >= MID_TERM_UPDATE_INTERVAL:
                    mid_term_2min_forecast = self.make_mid_term_forecast(current_price, 2)
                    self.current_forecast['mid_term_2min'] = mid_term_2min_forecast
                    self.last_mid_term_2min_update = now
                    predicted_event = 'upper' if mid_term_2min_forecast['prob_upper'] > mid_term_2min_forecast['prob_lower'] else 'lower'
                    self.mid_term_2min_pending_forecasts.append({
                        'timestamp': now,
                        'predicted_event': predicted_event,
                        'upper_target': mid_term_2min_forecast['upper_target'],
                        'lower_target': mid_term_2min_forecast['lower_target']
                    })

                # Проверка точности всех прогнозов
                self.check_accuracy()

        except Exception as e:
            print(f"Ошибка обработки сообщения: {str(e)}")

    def get_short_term_analysis(self, current_price):
        """Форматирование краткосрочного анализа с точностью."""
        short_term = self.current_forecast.get('short_term', {})
        accuracy = self.get_short_term_accuracy()
        output = [
            f"📈 Краткосрочный анализ {self.symbol} [{datetime.now().strftime('%H:%M:%S')}]",
            f"🏷 Текущая цена: {current_price:.2f} USD",
            "🧠 Прогноз на следующую секунду:",
            f"▴ Вероятность роста: {short_term.get('prob_up', 0.5)*100:.1f}%",
            f"▾ Вероятность падения: {short_term.get('prob_down', 0.5)*100:.1f}%",
            f"📊 Точность: {accuracy:.1f}%",
            f"📦 Записей в буфере: {len(self.market_data)}/{BUFFER_MAXLEN}"
        ]
        return "\n".join(output)

    def get_mid_term_1min_analysis(self, current_price):
        """Форматирование анализа на 1 минуту с точностью."""
        mid_term_1min = self.current_forecast.get('mid_term_1min', {})
        accuracy = self.get_mid_term_1min_accuracy()
        output = [
            f"📈 Среднесрочный анализ на 1 минуту {self.symbol} [{datetime.now().strftime('%H:%M:%S')}]",
            f"🏷 Текущая цена: {current_price:.2f} USD",
            "🧠 Прогноз на следующую минуту:",
            f"▴ Вероятность достижения {mid_term_1min.get('upper_target', current_price + 0.5):.2f} USD: {mid_term_1min.get('prob_upper', 0.5)*100:.1f}%",
            f"▾ Вероятность достижения {mid_term_1min.get('lower_target', current_price - 0.5):.2f} USD: {mid_term_1min.get('prob_lower', 0.5)*100:.1f}%",
            f"📊 Точность: {accuracy:.1f}%"
        ]
        return "\n".join(output)

    def get_mid_term_2min_analysis(self, current_price):
        """Форматирование анализа на 2 минуты с точностью."""
        mid_term_2min = self.current_forecast.get('mid_term_2min', {})
        accuracy = self.get_mid_term_2min_accuracy()
        output = [
            f"📈 Среднесрочный анализ на 2 минуты {self.symbol} [{datetime.now().strftime('%H:%M:%S')}]",
            f"🏷 Текущая цена: {current_price:.2f} USD",
            "🧠 Прогноз на следующие 2 минуты:",
            f"▴ Вероятность достижения {mid_term_2min.get('upper_target', current_price + 1):.2f} USD: {mid_term_2min.get('prob_upper', 0.5)*100:.1f}%",
            f"▾ Вероятность достижения {mid_term_2min.get('lower_target', current_price - 1):.2f} USD: {mid_term_2min.get('prob_lower', 0.5)*100:.1f}%",
            f"📊 Точность: {accuracy:.1f}%"
        ]
        return "\n".join(output)

    def print_all_analyses(self, current_price):
        """Вывод всех анализов в консоль."""
        clear_output(wait=True)
        short_term_output = self.get_short_term_analysis(current_price)
        mid_term_1min_output = self.get_mid_term_1min_analysis(current_price)
        mid_term_2min_output = self.get_mid_term_2min_analysis(current_price)
        full_output = f"{short_term_output}\n\n{mid_term_1min_output}\n\n{mid_term_2min_output}"
        print(full_output)

if __name__ == "__main__":
    analyzer = MarketAnalyzer("SOL-USDT")
    try:
        loop = asyncio.get_event_loop()
        loop.create_task(analyzer.maintain_connection())
        loop.run_forever()
    except KeyboardInterrupt:
        print("\nАнализ остановлен")
        if analyzer.ws:
            loop.run_until_complete(analyzer.ws.close())

📈 Краткосрочный анализ SOL-USDT [11:56:59]
🏷 Текущая цена: 133.39 USD
🧠 Прогноз на следующую секунду:
▴ Вероятность роста: 0.0%
▾ Вероятность падения: 100.0%
📊 Точность: 89.1%
📦 Записей в буфере: 3278/7000

📈 Среднесрочный анализ на 1 минуту SOL-USDT [11:56:59]
🏷 Текущая цена: 133.39 USD
🧠 Прогноз на следующую минуту:
▴ Вероятность достижения 133.93 USD: 52.3%
▾ Вероятность достижения 132.93 USD: 47.7%
📊 Точность: 20.0%

📈 Среднесрочный анализ на 2 минуты SOL-USDT [11:56:59]
🏷 Текущая цена: 133.39 USD
🧠 Прогноз на следующие 2 минуты:
▴ Вероятность достижения 134.43 USD: 52.3%
▾ Вероятность достижения 132.43 USD: 47.7%
📊 Точность: 0.0%
